# エアコン最適化システム - シンプル版

## 概要
シンプルで実行しやすいエアコン最適化システムです。
- クラス定義なしで直接実行
- 各ステップの結果をこまめに確認
- カラム名はそのまま使用
- カテゴリカル変数の変換ルールをJSONで事前定義


In [1]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import json
import os
import glob
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("📦 ライブラリ読み込み完了")
print("=" * 60)


📦 ライブラリ読み込み完了


In [2]:
# カテゴリカル変数の変換ルール読み込み
print("📋 設定ファイル読み込み開始")
print("=" * 60)

# カテゴリカル変数の変換ルールを読み込み
category_mapping_file = "category_mapping.json"
if os.path.exists(category_mapping_file):
    with open(category_mapping_file, "r", encoding="utf-8") as f:
        category_mapping = json.load(f)
    print(f"✅ カテゴリカル変換ルール読み込み完了")
    print(f"📋 変換ルール:")
    for col, mapping in category_mapping.items():
        print(f"   {col}: {mapping}")
else:
    print(f"❌ カテゴリカル変換ルールファイルが見つかりません: {category_mapping_file}")
    # デフォルトの変換ルールを設定
    category_mapping = {
        "A/C ON/OFF": {"OFF": 0, "ON": 1},
        "A/C Mode": {"Auto": 0, "Cool": 1, "Heat": 2, "Fan": 3, "Dry": 4},
        "A/C Fan Speed": {"Auto": 0, "Low": 1, "Medium": 2, "High": 3, "Quiet": 4}
    }
    print(f"⚠️ デフォルトの変換ルールを使用します")
    print(f"📋 デフォルト変換ルール:")
    for col, mapping in category_mapping.items():
        print(f"   {col}: {mapping}")

print(f"\n📊 設定ファイル読み込み結果:")
print(f"   カテゴリカル変換ルール: {len(category_mapping)}個の列")


📋 設定ファイル読み込み開始
✅ カテゴリカル変換ルール読み込み完了
📋 変換ルール:
   A/C ON/OFF: {'0': 0, '1': 1}
   A/C Mode: {'COOL': 1, 'FAN': 3, 'HEAT': 2}
   A/C Fan Speed: {'Auto': 0, 'High': 3, 'Low': 1, 'Medium': 2, 'Top': 4}

📊 設定ファイル読み込み結果:
   カテゴリカル変換ルール: 3個の列


In [3]:
# ストア名を指定
store_name = "Clea"
data_dir = f"data/{store_name}"

print(f"📁 {store_name}のデータ読み込み開始")
print("=" * 60)

# AC制御データの読み込み
print(f"📁 AC制御データ読み込み中...")
ac_files = glob.glob(f"{data_dir}/**/ac-control-*.csv", recursive=True)

if not ac_files:
    print("❌ AC制御データが見つかりません")
    ac_control_df = None
else:
    dfs = []
    for file in ac_files:
        try:
            df = pd.read_csv(file, encoding="utf-8")
            dfs.append(df)
            print(f"   ✅ {os.path.basename(file)}: {len(df):,}件")
        except Exception as e:
            print(f"   ❌ エラー {file}: {e}")
    
    if dfs:
        ac_control_df = pd.concat(dfs, ignore_index=True)
        print(f"   📊 合計: {len(ac_control_df):,}件")
        print(f"   📋 カラム: {list(ac_control_df.columns)}")
    else:
        ac_control_df = None

# 電力メーターデータの読み込み
print(f"\n📁 電力メーターデータ読み込み中...")
power_files = glob.glob(f"{data_dir}/**/ac-power-meter-*.csv", recursive=True)

if not power_files:
    print("❌ 電力メーターデータが見つかりません")
    power_meter_df = None
else:
    dfs = []
    for file in power_files:
        try:
            df = pd.read_csv(file, encoding="utf-8")
            dfs.append(df)
            print(f"   ✅ {os.path.basename(file)}: {len(df):,}件")
        except Exception as e:
            print(f"   ❌ エラー {file}: {e}")
    
    if dfs:
        power_meter_df = pd.concat(dfs, ignore_index=True)
        print(f"   📊 合計: {len(power_meter_df):,}件")
        print(f"   📋 カラム: {list(power_meter_df.columns)}")
    else:
        power_meter_df = None

# マスタデータの読み込み
print(f"\n📋 マスタデータ読み込み中...")
json_file = f"master/MASTER_{store_name}.json"
if os.path.exists(json_file):
    try:
        with open(json_file, "r", encoding="utf-8") as f:
            master_info = json.load(f)
        print(f"   ✅ JSONマスタデータ読み込み完了")
        print(f"   📋 キー: {list(master_info.keys())}")
    except Exception as e:
        print(f"   ❌ JSONマスタデータ読み込みエラー: {e}")
        master_info = None
else:
    print("   ⚠️ マスタデータが見つかりません")
    master_info = None

print(f"\n📊 データ読み込み結果:")
print(f"   AC制御データ: {ac_control_df.shape if ac_control_df is not None else 'None'}")
print(f"   電力メーターデータ: {power_meter_df.shape if power_meter_df is not None else 'None'}")
print(f"   マスタ情報: {list(master_info.keys()) if master_info else 'None'}")


📁 Cleaのデータ読み込み開始
📁 AC制御データ読み込み中...
   ✅ ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv: 34,560件
   ✅ ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv: 196,020件
   ✅ ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv: 196,702件
   ✅ ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv: 175,516件
   ✅ ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv: 196,416件
   ✅ ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-12-01-2024-12-31.csv: 196,416件
   ✅ ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-01-01-2025-01-31.csv: 196,438件
   ✅ ac-control-71,70,72,73-logs-2024-11-01-2024-11-30.csv: 34,560件
   ✅ ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-lo

In [4]:
# データ確認
print("📊 読み込みデータの確認")
print("=" * 60)

if ac_control_df is not None:
    print(f"📋 AC制御データ:")
    print(f"   形状: {ac_control_df.shape}")
    print(f"   カラム: {list(ac_control_df.columns)}")
    print(f"   最初の5行:")
    print(ac_control_df.head())
    print(f"   データ型:")
    print(ac_control_df.dtypes)
else:
    print("❌ AC制御データが読み込まれていません")

print("\n" + "="*60)

if power_meter_df is not None:
    print(f"📋 電力メーターデータ:")
    print(f"   形状: {power_meter_df.shape}")
    print(f"   カラム: {list(power_meter_df.columns)}")
    print(f"   最初の5行:")
    print(power_meter_df.head())
    print(f"   データ型:")
    print(power_meter_df.dtypes)
else:
    print("❌ 電力メーターデータが読み込まれていません")


📊 読み込みデータの確認
📋 AC制御データ:
   形状: (3400320, 13)
   カラム: ['A/C Name', 'Datetime', 'Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature', 'A/C ON/OFF', 'A/C Mode', 'A/C Fan Speed', 'Naive Energy Level', 'Airux Energy Level', 'Outdoor Room Temp.', 'Outdoor Set Temp.', 'Room Set Temp.']
   最初の5行:
  A/C Name                          Datetime  Outdoor Temp.  Indoor Temp.  \
0     G-24  2025-04-30 23:55:00.084567+09:00            6.0          16.0   
1     G-21  2025-04-30 23:55:00.084567+09:00            6.0          20.0   
2     G-23  2025-04-30 23:55:00.084567+09:00            6.0          16.0   
3     G-22  2025-04-30 23:55:00.084567+09:00            6.0          18.0   
4     G-23  2025-04-30 23:50:00.996488+09:00            6.0          16.0   

   A/C Set Temperature A/C ON/OFF A/C Mode A/C Fan Speed  Naive Energy Level  \
0                 26.0        OFF     COOL           NaN                   0   
1                 26.0        OFF     COOL           NaN                   0   
2    

In [5]:
# 出力ディレクトリの作成
output_dir = f"data_processed/{store_name}"
os.makedirs(output_dir, exist_ok=True)

print(f"🔧 データ前処理開始")
print("=" * 60)

# AC制御データの前処理
if ac_control_df is not None:
    print(f"🔧 AC制御データ前処理中...")
    
    # 日時列の特定と統一
    datetime_cols = [col for col in ac_control_df.columns if "datetime" in col.lower() or "日時" in col]
    if not datetime_cols:
        print("❌ 日時列が見つかりません")
        ac_control_processed = None
    else:
        datetime_col = datetime_cols[0]
        print(f"   📅 日時列: {datetime_col}")
        ac_control_df[datetime_col] = pd.to_datetime(ac_control_df[datetime_col], utc=True)
        ac_control_df[datetime_col] = ac_control_df[datetime_col].dt.tz_localize(None)
        ac_control_df[datetime_col] = ac_control_df[datetime_col].dt.floor("T")
        
        # 外れ値除去
        numeric_cols = ['Outdoor Temp.','Indoor Temp.','A/C Set Temperature']
        for col in numeric_cols:
            if col in ac_control_df.columns:
                before_count = len(ac_control_df)
                mean_val = ac_control_df[col].mean()
                std_val = ac_control_df[col].std()
                threshold = 3.0 * std_val
                ac_control_df = ac_control_df[abs(ac_control_df[col] - mean_val) <= threshold]
                after_count = len(ac_control_df)
                print(f"   🔄 {col}外れ値除去: {before_count:,} → {after_count:,}件")
        
        # 線形補間による欠損値処理
        for col in numeric_cols:
            if col in ac_control_df.columns:
                before_nan = ac_control_df[col].isna().sum()
                ac_control_df[col] = ac_control_df[col].interpolate(method='linear')
                after_nan = ac_control_df[col].isna().sum()
                print(f"   🔄 {col}欠損値補完: {before_nan:,} → {after_nan:,}件")
        
        # カテゴリカル変数の処理（JSON定義を使用）
        categorical_cols = ['A/C ON/OFF','A/C Mode','A/C Fan Speed']
        for col in categorical_cols:
            if col in ac_control_df.columns and col in category_mapping:
                # 変換前の値を確認（NaN値を除外）
                unique_values = ac_control_df[col].dropna().unique()
                print(f"   📋 {col}変換前の値: {sorted(unique_values)}")
                
                # 定義された変換ルールを適用
                mapping = category_mapping[col]
                ac_control_df[col] = ac_control_df[col].map(mapping)
                
                # 未定義の値がある場合は警告
                unmapped_values = ac_control_df[col].isna().sum()
                if unmapped_values > 0:
                    print(f"   ⚠️ {col}で未定義の値が{unmapped_values}件あります")
                    # 未定義の値は-1で埋める
                    ac_control_df[col] = ac_control_df[col].fillna(-1)
                
                print(f"   🔄 {col}変換ルール: {mapping}")
                print(f"   ✅ {col}カテゴリ変換完了")
            elif col in ac_control_df.columns:
                print(f"   ⚠️ {col}の変換ルールが定義されていません")
        
        # 日付列の追加
        ac_control_df["date"] = ac_control_df[datetime_col].dt.date
        ac_control_df["datetime"] = ac_control_df[datetime_col]
        
        ac_control_processed = ac_control_df
        print(f"   ✅ AC制御データ前処理完了: {len(ac_control_processed):,}件")
        print(f"   📋 最終カラム: {list(ac_control_processed.columns)}")
else:
    ac_control_processed = None
    print("   ⚠️ AC制御データが存在しないため、前処理をスキップします")

# 電力メーターデータの前処理
if power_meter_df is not None:
    print(f"\n🔧 電力メーターデータ前処理中...")
    
    # 日時列の特定と統一
    datetime_cols = [col for col in power_meter_df.columns if "datetime" in col.lower() or "日時" in col]
    if not datetime_cols:
        print("❌ 日時列が見つかりません")
        power_meter_processed = None
    else:
        datetime_col = datetime_cols[0]
        print(f"   📅 日時列: {datetime_col}")
        power_meter_df[datetime_col] = pd.to_datetime(power_meter_df[datetime_col], utc=True)
        power_meter_df[datetime_col] = power_meter_df[datetime_col].dt.tz_localize(None)
        power_meter_df[datetime_col] = power_meter_df[datetime_col].dt.floor("T")
        
        # 外れ値除去
        if "Phase A" in power_meter_df.columns:
            before_count = len(power_meter_df)
            mean_val = power_meter_df["Phase A"].mean()
            std_val = power_meter_df["Phase A"].std()
            threshold = 3.0 * std_val
            power_meter_df = power_meter_df[abs(power_meter_df["Phase A"] - mean_val) <= threshold]
            after_count = len(power_meter_df)
            print(f"   🔄 Phase A外れ値除去: {before_count:,} → {after_count:,}件")
        
        # 欠損値処理
        if "Phase A" in power_meter_df.columns:
            before_nan = power_meter_df["Phase A"].isna().sum()
            power_meter_df["Phase A"] = power_meter_df["Phase A"].fillna(0)
            after_nan = power_meter_df["Phase A"].isna().sum()
            print(f"   🔄 Phase A欠損値補完: {before_nan:,} → {after_nan:,}件")
        
        # 日付列の追加
        power_meter_df["date"] = power_meter_df[datetime_col].dt.date
        power_meter_df["datetime"] = power_meter_df[datetime_col]
        
        power_meter_processed = power_meter_df
        print(f"   ✅ 電力メーターデータ前処理完了: {len(power_meter_processed):,}件")
        print(f"   📋 最終カラム: {list(power_meter_processed.columns)}")
else:
    power_meter_processed = None
    print("   ⚠️ 電力メーターデータが存在しないため、前処理をスキップします")

# 前処理済みデータの保存
if ac_control_processed is not None:
    ac_path = f"{output_dir}/ac_control_processed_{store_name}.csv"
    ac_control_processed.to_csv(ac_path, index=False, encoding="utf-8-sig")
    print(f"\n💾 AC制御データ保存: {ac_path}")

if power_meter_processed is not None:
    power_path = f"{output_dir}/power_meter_processed_{store_name}.csv"
    power_meter_processed.to_csv(power_path, index=False, encoding="utf-8-sig")
    print(f"💾 電力メーターデータ保存: {power_path}")

print(f"\n📊 データ前処理結果:")
print(f"   AC制御データ: {ac_control_processed.shape if ac_control_processed is not None else 'None'}")
print(f"   電力メーターデータ: {power_meter_processed.shape if power_meter_processed is not None else 'None'}")


🔧 データ前処理開始
🔧 AC制御データ前処理中...
   📅 日時列: Datetime
   🔄 Outdoor Temp.外れ値除去: 3,400,320 → 3,400,112件
   🔄 Indoor Temp.外れ値除去: 3,400,112 → 3,322,916件
   🔄 A/C Set Temperature外れ値除去: 3,322,916 → 3,309,404件
   🔄 Outdoor Temp.欠損値補完: 0 → 0件
   🔄 Indoor Temp.欠損値補完: 0 → 0件
   🔄 A/C Set Temperature欠損値補完: 0 → 0件
   📋 A/C ON/OFF変換前の値: ['OFF', 'ON']
   ⚠️ A/C ON/OFFで未定義の値が3309404件あります
   🔄 A/C ON/OFF変換ルール: {'0': 0, '1': 1}
   ✅ A/C ON/OFFカテゴリ変換完了
   📋 A/C Mode変換前の値: ['COOL', 'FAN', 'HEAT']
   ⚠️ A/C Modeで未定義の値が44172件あります
   🔄 A/C Mode変換ルール: {'COOL': 1, 'FAN': 3, 'HEAT': 2}
   ✅ A/C Modeカテゴリ変換完了
   📋 A/C Fan Speed変換前の値: ['Auto', 'High', 'Low', 'Medium', 'Top']
   ⚠️ A/C Fan Speedで未定義の値が2331156件あります
   🔄 A/C Fan Speed変換ルール: {'Auto': 0, 'High': 3, 'Low': 1, 'Medium': 2, 'Top': 4}
   ✅ A/C Fan Speedカテゴリ変換完了
   ✅ AC制御データ前処理完了: 3,309,404件
   📋 最終カラム: ['A/C Name', 'Datetime', 'Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature', 'A/C ON/OFF', 'A/C Mode', 'A/C Fan Speed', 'Naive Energy Level', 'Airux Energy Le

In [6]:
# 前処理結果の確認
print("📊 前処理結果の確認")
print("=" * 60)

if ac_control_processed is not None:
    print(f"📋 AC制御データ（前処理後）:")
    print(f"   形状: {ac_control_processed.shape}")
    print(f"   最初の5行:")
    print(ac_control_processed.head())
    print(f"   欠損値数:")
    print(ac_control_processed.isnull().sum())
else:
    print("❌ AC制御データが前処理されていません")

print("\n" + "="*60)

if power_meter_processed is not None:
    print(f"📋 電力メーターデータ（前処理後）:")
    print(f"   形状: {power_meter_processed.shape}")
    print(f"   最初の5行:")
    print(power_meter_processed.head())
    print(f"   欠損値数:")
    print(power_meter_processed.isnull().sum())
else:
    print("❌ 電力メーターデータが前処理されていません")


📊 前処理結果の確認
📋 AC制御データ（前処理後）:
   形状: (3309404, 15)
   最初の5行:
  A/C Name            Datetime  Outdoor Temp.  Indoor Temp.  \
0     G-24 2025-04-30 14:55:00            6.0          16.0   
1     G-21 2025-04-30 14:55:00            6.0          20.0   
2     G-23 2025-04-30 14:55:00            6.0          16.0   
3     G-22 2025-04-30 14:55:00            6.0          18.0   
4     G-23 2025-04-30 14:50:00            6.0          16.0   

   A/C Set Temperature  A/C ON/OFF  A/C Mode  A/C Fan Speed  \
0                 26.0        -1.0       1.0           -1.0   
1                 26.0        -1.0       1.0           -1.0   
2                 22.0        -1.0       2.0           -1.0   
3                 22.0        -1.0       2.0            2.0   
4                 22.0        -1.0       2.0           -1.0   

   Naive Energy Level  Airux Energy Level  Outdoor Room Temp.  \
0                   0                   0               -10.0   
1                   0                   0            

In [11]:
# 特徴量作成
print("🔧 特徴量作成開始")
print("=" * 60)

if ac_control_processed is not None and power_meter_processed is not None:
    print("📊 時別特徴量データの作成中...")
    
    # 日時範囲の取得
    start_date = ac_control_processed['datetime'].min()
    end_date = ac_control_processed['datetime'].max()
    print(f"   📅 期間: {start_date} ～ {end_date}")
    
    # AC制御データの時別集計
    print("   🔄 AC制御データの時別集計中...")
    ac_hourly = ac_control_processed.groupby('datetime').agg({
        'Outdoor Temp.': ['mean', 'std', 'min', 'max'],
        'Indoor Temp.': ['mean', 'std', 'min', 'max'],
        'A/C Set Temperature': ['mean', 'std', 'min', 'max']
    }).reset_index()
    
    # カラム名を平坦化
    ac_hourly.columns = ['datetime'] + [f"{col[0]}_{col[1]}" for col in ac_hourly.columns[1:]]
    
    # 電力データの時別集計
    print("   🔄 電力データの時別集計中...")
    power_hourly = power_meter_processed.groupby('datetime').agg({
        'Phase A': ['mean', 'std', 'min', 'max', 'sum']
    }).reset_index()
    
    # カラム名を平坦化
    power_hourly.columns = ['datetime'] + [f"power_{col[1]}" for col in power_hourly.columns[1:]]
    
    # データをマージ
    print("   🔄 データマージ中...")
    feature_df = pd.merge(ac_hourly, power_hourly, on='datetime', how='outer')
    
    # 時間特徴量の追加
    feature_df['date'] = feature_df['datetime'].dt.date
    feature_df['hour'] = feature_df['datetime'].dt.hour
    feature_df['day_of_week'] = feature_df['datetime'].dt.weekday
    feature_df['is_weekend'] = feature_df['day_of_week'] >= 5
    feature_df['is_business_hours'] = (feature_df['hour'] >= 9) & (feature_df['hour'] <= 18)
    
    # ラグ特徴量の追加
    print("   🔄 ラグ特徴量の追加中...")
    lag_cols = ['Outdoor Temp._mean', 'Indoor Temp._mean', 'A/C Set Temperature_mean']
    for col in lag_cols:
        if col in feature_df.columns:
            feature_df[f"{col}_lag1"] = feature_df[col].shift(1)
    
    # 欠損値の処理
    print("   🔄 欠損値処理中...")
    feature_df = feature_df.fillna(0)
    
    print(f"   ✅ 特徴量作成完了: {len(feature_df):,}件")
    print(f"   📋 特徴量カラム: {list(feature_df.columns)}")
    
    # 欠損値の確認
    print(f"   📊 欠損値数:")
    print(feature_df.isnull().sum())
    
else:
    feature_df = None
    print("   ⚠️ 前処理済みデータが存在しないため、特徴量作成をスキップします")

print(f"\n📊 特徴量作成結果:")
print(f"   特徴量データ: {feature_df.shape if feature_df is not None else 'None'}")

🔧 特徴量作成開始
📊 時別特徴量データの作成中...
   📅 期間: 2024-06-30 15:00:00 ～ 2025-09-28 01:25:00
   🔄 AC制御データの時別集計中...
   🔄 電力データの時別集計中...
   🔄 データマージ中...
   🔄 ラグ特徴量の追加中...
   🔄 欠損値処理中...
   ✅ 特徴量作成完了: 649,809件
   📋 特徴量カラム: ['datetime', 'Outdoor Temp._mean', 'Outdoor Temp._std', 'Outdoor Temp._min', 'Outdoor Temp._max', 'Indoor Temp._mean', 'Indoor Temp._std', 'Indoor Temp._min', 'Indoor Temp._max', 'A/C Set Temperature_mean', 'A/C Set Temperature_std', 'A/C Set Temperature_min', 'A/C Set Temperature_max', 'power_mean', 'power_std', 'power_min', 'power_max', 'power_sum', 'date', 'hour', 'day_of_week', 'is_weekend', 'is_business_hours', 'Outdoor Temp._mean_lag1', 'Indoor Temp._mean_lag1', 'A/C Set Temperature_mean_lag1']
   📊 欠損値数:
datetime                         0
Outdoor Temp._mean               0
Outdoor Temp._std                0
Outdoor Temp._min                0
Outdoor Temp._max                0
Indoor Temp._mean                0
Indoor Temp._std                 0
Indoor Temp._min                

In [8]:
# 特徴量データの確認
print("📊 特徴量データの確認")
print("=" * 60)

if feature_df is not None:
    print(f"📋 特徴量データ:")
    print(f"   形状: {feature_df.shape}")
    print(f"   最初の5行:")
    print(feature_df.head())
    print(f"   データ型:")
    print(feature_df.dtypes)
    print(f"   基本統計量:")
    print(feature_df.describe())
else:
    print("❌ 特徴量データが作成されていません")


📊 特徴量データの確認
📋 特徴量データ:
   形状: (649809, 26)
   最初の5行:
             datetime  Outdoor Temp._mean  Outdoor Temp._std  \
0 2024-06-30 15:00:00                22.0                0.0   
1 2024-06-30 15:05:00                22.0                0.0   
2 2024-06-30 15:10:00                22.0                0.0   
3 2024-06-30 15:15:00                22.0                0.0   
4 2024-06-30 15:20:00                22.0                0.0   

   Outdoor Temp._min  Outdoor Temp._max  Indoor Temp._mean  Indoor Temp._std  \
0               22.0               22.0          27.576923          1.270372   
1               22.0               22.0          27.576923          1.270372   
2               22.0               22.0          27.500000          1.272792   
3               22.0               22.0          27.423077          1.270372   
4               22.0               22.0          27.423077          1.301478   

   Indoor Temp._min  Indoor Temp._max  A/C Set Temperature_mean  ...  \
0         

In [9]:
# 機械学習ライブラリのインポート
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import joblib

print("🤖 機械学習ライブラリ読み込み完了")
print("=" * 60)

# モデル訓練用ディレクトリの作成
models_dir = f"models/{store_name}"
os.makedirs(models_dir, exist_ok=True)

print(f"📁 モデル保存ディレクトリ: {models_dir}")

# 特徴量データの準備
if feature_df is not None:
    print("📊 モデル訓練用データの準備中...")
    
    # 欠損値を含む行を除去
    feature_df_clean = feature_df.dropna()
    print(f"   🔄 欠損値除去: {len(feature_df):,} → {len(feature_df_clean):,}件")
    
    # 特徴量とターゲットの分離
    feature_columns = [col for col in feature_df_clean.columns if col not in ['datetime', 'date']]
    X = feature_df_clean[feature_columns]
    
    print(f"   📋 特徴量数: {len(feature_columns)}")
    print(f"   📋 特徴量名: {feature_columns}")
    
    # データの分割（訓練:80%, テスト:20%）
    X_train, X_test, y_train, y_test = train_test_split(
        X, X, test_size=0.2, random_state=42, shuffle=False
    )
    
    print(f"   📊 訓練データ: {X_train.shape}")
    print(f"   📊 テストデータ: {X_test.shape}")
    
    print("   ✅ モデル訓練準備完了")
else:
    print("   ⚠️ 特徴量データが存在しないため、モデル訓練準備をスキップします")


🤖 機械学習ライブラリ読み込み完了
📁 モデル保存ディレクトリ: models/Clea
📊 モデル訓練用データの準備中...
   🔄 欠損値除去: 649,809 → 649,809件
   📋 特徴量数: 24
   📋 特徴量名: ['Outdoor Temp._mean', 'Outdoor Temp._std', 'Outdoor Temp._min', 'Outdoor Temp._max', 'Indoor Temp._mean', 'Indoor Temp._std', 'Indoor Temp._min', 'Indoor Temp._max', 'A/C Set Temperature_mean', 'A/C Set Temperature_std', 'A/C Set Temperature_min', 'A/C Set Temperature_max', 'power_mean', 'power_std', 'power_min', 'power_max', 'power_sum', 'hour', 'day_of_week', 'is_weekend', 'is_business_hours', 'Outdoor Temp._mean_lag1', 'Indoor Temp._mean_lag1', 'A/C Set Temperature_mean_lag1']
   📊 訓練データ: (519847, 24)
   📊 テストデータ: (129962, 24)
   ✅ モデル訓練準備完了


In [10]:
# 実行結果サマリー
print("📊 実行結果サマリー")
print("=" * 60)

print(f"🏪 ストア名: {store_name}")
print(f"📁 データディレクトリ: {data_dir}")
print(f"💾 出力ディレクトリ: {output_dir}")
print(f"🤖 モデルディレクトリ: {models_dir}")

print(f"\n📊 データ処理結果:")
print(f"   AC制御データ: {ac_control_df.shape if ac_control_df is not None else 'None'}")
print(f"   電力メーターデータ: {power_meter_df.shape if power_meter_df is not None else 'None'}")
print(f"   前処理済みAC制御データ: {ac_control_processed.shape if ac_control_processed is not None else 'None'}")
print(f"   前処理済み電力メーターデータ: {power_meter_processed.shape if power_meter_processed is not None else 'None'}")
print(f"   特徴量データ: {feature_df.shape if feature_df is not None else 'None'}")

if feature_df is not None:
    print(f"\n📋 特徴量情報:")
    print(f"   特徴量数: {len(feature_columns) if 'feature_columns' in locals() else 'None'}")
    print(f"   データ期間: {feature_df['datetime'].min()} ～ {feature_df['datetime'].max()}")
    print(f"   時間数: {len(feature_df):,}時間")

print(f"\n✅ シンプル版エアコン最適化システムの基本処理が完了しました")
print(f"   次のステップ: モデル訓練、予測、最適化の実装")


📊 実行結果サマリー
🏪 ストア名: Clea
📁 データディレクトリ: data/Clea
💾 出力ディレクトリ: data_processed/Clea
🤖 モデルディレクトリ: models/Clea

📊 データ処理結果:
   AC制御データ: (3309404, 15)
   電力メーターデータ: (12920652, 8)
   前処理済みAC制御データ: (3309404, 15)
   前処理済み電力メーターデータ: (12920652, 8)
   特徴量データ: (649809, 26)

📋 特徴量情報:
   特徴量数: 24
   データ期間: 2024-06-30 15:00:00 ～ 2025-09-28 01:25:00
   時間数: 649,809時間

✅ シンプル版エアコン最適化システムの基本処理が完了しました
   次のステップ: モデル訓練、予測、最適化の実装
